In [1]:
import snow.utils as sf
import pandas as pd
from fbprophet import Prophet

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [6]:
financials_df = sf.from_snow(wh='load_wh'
                             ,role='all_data_viewer'
                             ,db='all_data'
                             ,q_kind='user_financials'
                             ,to_df=True)

Using all_data_viewer...
Established connection.
Querying load_wh.all_data.postgres
User_financials data:  ('2018-06-23', 'Gong Cha', Decimal('10.000000'), '113205', 11.9, 8.0, 1, 408, 'Melbourne', 11.9, 19.9, 19.9) 

Connection succesfully closed.


In [21]:
## Convert strings to datetime objects
financials_df.ds = pd.to_datetime(financials_df.ds)
financials_df.head()

,ds,merchant,rate,uid,lifetime_transaction_count,lifetime_organic_volume,lifetime_promo_volume,sign_up_delta,city,transaction_org_volume,lifetime_aov,transaction_total_volume
0,2018-06-23,Gong Cha,10.000000,113205,11.90,8.0,1,408,Melbourne,11.9,19.9,19.9
1,2018-11-02,Gong Cha,10.000000,149209,2267.79,6.4,170,341,Melbourne,14.4,7.2,14.4
2,2019-03-02,CoCo Fresh Tea & Juice,10.000000,254227,412.00,0.0,33,205,Sydney,12.5,12.5,12.5
3,2018-12-08,CoCo Fresh Tea & Juice,15.000000,224410,415.43,7.6,24,239,Sydney,15.0,15.0,15.0
4,2018-07-09,8Bit,20.000000,136073,0.50,12.0,1,370,Melbourne,0.5,12.5,12.5


### Feature engineering

Now need to derive a

    -> last transaction delta; [tick]
    
    -> repeat transaction delta; [tick]
    
    -> progressive single merchant volume; [tick]
    
    -> progressive organic volume count; [tick]
    
    -> progressive transaction count; [tick]
    
    -> promo volume for single transactions; [tick]
    
    -> progressive merchant count; [tick]
    
    -> progressive geo_cluster count; [not yet resolved!!]
    
    -> progressive merchant transaction count. [tick]
    
    ...   

In [73]:
financials_df.shape[0]

564272

In [ ]:
transaction_delta = []
repeat_transaction_delta = []

progressive_transaction_count = []
progressive_organic_volume = []

progressive_merchant_count = []

progressive_merchant_volume = []
progressive_merchant_transaction_count = []

single_transaction_promo_volume = []

financials_df = financials_df.drop_duplicates().sort_values(by='ds',ascending=False)

df = financials_df.copy()

count=0
for i,r in df.iterrows():
    count+=1
    uid = r.uid; merchant = r.merchant; date = r.ds
    
    user_df = df[df.uid==uid]
    prior_df = user_df[user_df.ds<=date]
    shaved_df = prior_df[prior_df.index!=i]
    
    if count % 5000 == 0:
        print(str(df.shape[0]-count)+' transactions left to process...')
    
    single_transaction_promo_volume += [r.transaction_total_volume - r.transaction_org_volume]

    progressive_transaction_count += [prior_df.shape[0]]
    progressive_merchant_count += [prior_df.merchant.nunique()]
    progressive_organic_volume += [prior_df.transaction_org_volume.sum()]
    
    if shaved_df.shape[0]>0:
        transaction_delta += [(date-list(shaved_df.ds)[0]).days]
    else:
        transaction_delta += [None]
    
    merchant_df = shaved_df[shaved_df.merchant==merchant]
    
    if merchant_df.shape[0]>0:
        repeat_transaction_delta += [(date-list(merchant_df.ds)[0]).days]
    else:
        repeat_transaction_delta += [None]
        
    progressive_merchant_volume += [(merchant_df.transaction_org_volume.sum())]
    progressive_merchant_transaction_count += [merchant_df.shape[0]+1]
    
    
    
financials_df['transaction_promo_volume'] = single_transaction_promo_volume
financials_df['prog_transaction_count'] = progressive_transaction_count
financials_df['prog_merchant_count'] = progressive_merchant_count
financials_df['prog_merchant_volume'] = progressive_merchant_volume
financials_df['prog_merchant_transaction_count'] = progressive_merchant_transaction_count
financials_df['prog_organic_volume'] = progressive_organic_volume
financials_df['repeat_transaction_delta'] = repeat_transaction_delta
financials_df['transaction_delta'] = transaction_delta

559272 transactions left to process...
554272 transactions left to process...
549272 transactions left to process...
544272 transactions left to process...
539272 transactions left to process...
534272 transactions left to process...
529272 transactions left to process...
524272 transactions left to process...
519272 transactions left to process...
514272 transactions left to process...
509272 transactions left to process...
504272 transactions left to process...
499272 transactions left to process...
494272 transactions left to process...
489272 transactions left to process...
484272 transactions left to process...
479272 transactions left to process...
474272 transactions left to process...
469272 transactions left to process...
464272 transactions left to process...
459272 transactions left to process...
454272 transactions left to process...
449272 transactions left to process...
444272 transactions left to process...
439272 transactions left to process...
434272 transactions left 

In [58]:
progressive_merchant_volume

[229.35]

In [60]:
progressive_merchant_transaction_count

[36]

In [ ]:
single